In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import numpy as np
import random

In [21]:
# Set up Spotify API credentials
CLIENT_ID = "f3250f9c769c472d882b669d189f2d3f"
CLIENT_SECRET = "b7be4f9fc76f4ab29242c7383cbeac09"
REDIRECT_URI = 'http://localhost:8889/callback'
# Specify the desired scopes, including 'playlist-read-private'
SCOPE = 'playlist-read-private'

# Authenticate with the Spotify API using OAuth2
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=SCOPE))

In [40]:
#Extracted mood features for multiple related playlists
moods = {
    'Happy': {
        'loudness': (-6.6071,-3.5099),
        'speechiness': (0.0248, 0.2248),
        'acousticness': (6.89e-05, 0.3300689),
        'instrumentalness': (0.0, 0.001),
        'liveness': (0.0347, 0.2348),
        'tempo': (72.38, 172.38),
        'valence': (0.2982, 0.8982),
        'energy': (0.508, 0.91),
        'danceability': (0.618, 0.818)
    },
    'Sad': {
        'loudness': (-22.8234,-2.8224),
        'speechiness': (0.0241, 0.2241),
        'acousticness': (0, 1),
        'instrumentalness': (0.0, 0.02),
        'liveness': (0.0386, 0.2386),
        'tempo': (76.279, 146.27899),
        'valence': (0, 0.476),
        'energy': (0, 1),
        'danceability': (0.418, 0.718)
    },
    'Depressing': {
        'loudness': (-19.559,-7.56),
        'speechiness': (0.0231, 0.0732),
        'acousticness': (0, 0.81),
        'instrumentalness': (0.0, 0.1),
        'liveness': (0.0552, 0.3668),
        'tempo': (97.895,147.5),
        'valence': (0.0959, 0.5194),
        'energy': (0.2147, 0.7214),
        'danceability': (0.426, 0.726)
    },
    'Heartbroken': {
        'loudness': (-15.414,-3.61),
        'speechiness': (0.0261, 0.097),
        'acousticness': (0.000216, 0.800216),
        'instrumentalness': (0.0, 0.1),
        'liveness': (0.0522, 0.2523),
        'tempo': (75.5,150.5),
        'valence': (0.07, 0.5691),
        'energy': (0.2164, 0.816401),
        'danceability': (0.375, 0.774001)
    },
    'Amusing': {
        'loudness': (-15.3,-2.4),
        'speechiness': (0.026, 0.086),
        'acousticness': (3.84e-05, 0.5000384),
        'instrumentalness': (0.0, 0.1),
        'liveness': (0.041, 0.241),
        'tempo': (70.13,145.13),
        'valence': (0.16899, 0.769),
        'energy': (0.4, 0.9),
        'danceability': (0.384, 0.784)
    },
    'Annoying': {
        'loudness': (-14.8856,-2.8826),
        'speechiness': (2.34e-05, 0.500023),
        'acousticness': (0.0, 0.1),
        'instrumentalness': (0.0538, 0.3539),
        'liveness': (0.130979, 0.63098),
        'tempo': (91.414,196.445),
        'valence': (0.350, 0.951),
        'energy': (0.543, 0.944),
        'danceability': (0.543, 0.944)
    },
    'Anxious': {
        'loudness': (-28.9302,-16.9302),
        'speechiness': (0.03581, 0.0458),
        'acousticness': (0.693, 0.994),
        'instrumentalness': (0.546, 0.9461),
        'liveness': (0.0815, 0.1815),
        'tempo': (58.3029,103.302995),
        'valence': (0.0319, 0.1319),
        'energy': (0.00245, 0.20245),
        'danceability': (0.0607, 0.2607)
    },
    'Beautiful': {
        'loudness': (-23.1,-4.885),
        'speechiness': (0.27, 0.474),
        'acousticness': (0.490, 0.99),
        'instrumentalness': (0, 1),
        'liveness': (0.04, 0.14),
        'tempo': (69.8,144.8),
        'valence': (0.0311, 0.3312),
        'energy': (0.00335, 0.50335),
        'danceability': (0.0818, 0.5818)
    },
    'Romantic': {
        'loudness': (-13.39902,-7.3992),
        'speechiness': (0.275, 0.575),
        'acousticness': (0.1202, 0.8202),
        'instrumentalness': (0, 1),
        'liveness': (0.0427, 0.2427),
        'tempo': (80.6699,155.660),
        'valence': (0.526999, 0.92699),
        'energy': (0.386, 0.7860),
        'danceability': (0.389, 0.69)
    },
    'Relaxing': {
        'loudness':(-21.62,-3.623),
        'speechiness': (0.245, 0.845),
        'acousticness': (0.090069, 0.990069),
        'instrumentalness': (0, 0.9),
        'liveness': (0.05189, 0.2519),
        'tempo': (73.4519,148.4519),
        'valence': (0.02, 0.52),
        'energy': (0.07204, 0.7720),
        'danceability': (0.41309, 0.81309)
    },
    'Dreamy': {
        'loudness':(-17.167,-5.16),
        'speechiness': (0.253, 0.553),
        'acousticness': (7.53e-06, 0.8),
        'instrumentalness': (0, 0.600),
        'liveness': (0.0655, 0.1655),
        'tempo': (76.05395,151.0541),
        'valence': (0.0601, 0.4601),
        'energy': (0.1803, 0.6803),
        'danceability': (0.309, 0.71)
    },
    'Energize': {
        'loudness': (-13.176000000000002,-1.1),
        'speechiness': (0,1),
        'acousticness': (1.95e-06, 0.300195),
        'instrumentalness': (0, 0.8),
        'liveness': (0.051, 0.3511),
        'tempo': (86.449999,161.51),
        'valence': (0.025, 0.625),
        'energy': (0.592, 0.992),
        'danceability': (0.5418, 0.8418)
    },
    'Erotic': {
        'loudness': (-59.928,-40.928),
        'speechiness': (0,1),
        'acousticness': (0, 1),
        'instrumentalness': (0,1),
        'liveness': (0.0461, 0.2461),
        'tempo': (90.544999,150.5537),
        'valence': (0.298, 0.798),
        'energy': (0.41, 0.81),
        'danceability': (0.659, 0.9591)
    },
    'Fearful': {
        'loudness': (-21.4,-3.3),
        'speechiness': (0.257, 0.757),
        'acousticness': (0,1),
        'instrumentalness': (0, 0.8),
        'liveness': (0.0603, 0.3603),
        'tempo': (80.375,140.378),
        'valence': (0.0358, 0.7358),
        'energy': (0.14532, 0.74531),
        'danceability': (0.2915, 0.692)
    },
    'Heroic': {
        'loudness': (-10.144,-4.1419),
        'speechiness': (0,1),
        'acousticness': (0.00064, 0.100641),
        'instrumentalness': (0, 0.1),
        'liveness': (0.0621, 0.16212),
        'tempo': (102.02405,177.023),
        'valence': (0.0412, 0.641001),
        'energy': (0.5662, 0.867),
        'danceability': (0.342, 0.74)
    }
}

In [41]:
def get_data_frame(year):
    search_phrase='year:' + str(year)
    x = random.randint(0,1000)
    results = sp.search(q=search_phrase, type='track', limit=50,offset=x)
    track_ids=[]

    for track in results['tracks']['items']:
        track_ids.append(track['id'])
    # Create an empty list to store feature dictionaries
    feature_dicts = []

    # Create an empty list to store feature dictionaries
    feature_dicts = []

    # Iterate through the track IDs
    for track_id in track_ids:
        # Get the audio features for the track
        audio_features = sp.audio_features(track_id)[0]

        # Get track details
        track_details = sp.track(track_id)

        # Create a dictionary with the audio features
        features_dict = {
            'Track Name': track_details['name'],
            'Artist': ', '.join([artist['name'] for artist in track_details['artists']]),
            'Duration (ms)': track_details['duration_ms'],
            'loudness': audio_features['loudness'],
            'speechiness': audio_features['speechiness'],
            'acousticness': audio_features['acousticness'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'tempo': audio_features['tempo'],
            'valence': audio_features['valence'],
            'energy': audio_features['energy'],
            'danceability': audio_features['danceability']
        }

        feature_dicts.append(features_dict)
    df = pd.DataFrame(feature_dicts)
    return df


In [42]:
def get_song_from_mood(mood,df):
    # Specify the mood you want to check
    #mood_name = 'heartbroken'
    mood_name = mood
    mood_features = moods[mood_name]
    #print(mood_features)
    song_list=[]
    # Check if the track's features match the mood's feature ranges
    for i in range(len(df)):
        track = df.iloc[i]  # Get the first track from the DataFrame
        is_matching = all(min_value <= track[feature] <= max_value for feature, (min_value, max_value) in mood_features.items())

        # Print the result
        
        if is_matching:
            print(f"Track- {track['Track Name']} by {track['Artist']} ")
            song=f"Track- {track['Track Name']} by {track['Artist']}"
            song_list.append(song)
            #print(song_list)
        else:
            pass
            #print(f"The track '{track['Track Name']}' by {track['Artist']} does not match the '{mood_name}' mood.")
    return song_list

In [43]:
import random

#code for One emotion
song_list=[]
while(len(song_list)==0):
    year=2010
    selected_choice='Happy'
    df=get_data_frame(year)
    song_list=get_song_from_mood(selected_choice,df)

In [44]:
print(len(df))

50


In [45]:
from PIL import ImageTk, Image
import customtkinter as ctk
import pyperclip
import tkinter
from tkinter import filedialog
import random
ctk.set_appearance_mode("dark")

root=ctk.CTk()
root.title("Get text")
root.geometry("800x600")

def on_close():
    global is_window_closed
    is_window_closed = True
    root.destroy()

global m,n

#RADIOBUTTON    
def select_year():
    print(radio_var.get())
    dialog = ctk.CTkInputDialog(text="Type in a year (1960-2023):", title="ENTER YEAR",
                               button_fg_color="#EE653C",button_hover_color="#F66E45")
    global m
    m = dialog.get_input()  
    print(m,radio_var.get())
    
def dont_select_year():
    global n
    n = random.randint(1970,2022)
    print(n,radio_var.get())
    
radio_var = tkinter.IntVar(value=2)
radiobutton_1 = ctk.CTkRadioButton(root, text="Select Year(1960-2023)",
                                             command=select_year, variable= radio_var, value=1,
                                             fg_color="#FC4A68",hover_color="#FC3E5E")
radiobutton_1.place(relx=0.2, rely=0.1, anchor=tkinter.CENTER)
radiobutton_2 = ctk.CTkRadioButton(root, text="Dont Select Year",
                                             command=dont_select_year, variable= radio_var, value=2,
                                             fg_color="#F1F648",hover_color="#F3F92C")
radiobutton_2.place(relx=0.175, rely=0.2, anchor=tkinter.CENTER)


#OPTION MENU

def optionmenu_callback(choice):
    global selected_choice
    selected_choice = choice
    print("optionmenu dropdown clicked:",choice)

optionmenu = ctk.CTkOptionMenu(root, values=["Happy","Sad","Depressing","Heartbroken","Amusing",
                                             "Annoying","Anxious","Beautiful","Romantic","Relaxing",
                                             "Dreamy","Energize","Erotic","Fearful","Heroic"],
                                command=optionmenu_callback,
                                fg_color=("#EE653C"),button_color=("#EE653C"),
                                button_hover_color=("#F66E45"))
optionmenu.set("Happy")
optionmenu.place(relx=0.7, rely=0.15, anchor=tkinter.CENTER)


#SEARCH SONG BUTTON
def button_event():
    #year
    global m,n
    if(radio_var.get()==2):
        year=int(n)
    else:
        year=int(m)
    #emotion
    global selected_choice
    print("button pressed")
    print(year,selected_choice)
    
    song_list=[]
    while(len(song_list)==0):
        df=get_data_frame(year)
        song_list=get_song_from_mood(selected_choice,df)
    print(len(song_list))
    textbox.delete("0.0", "end")
    for i in range(len(song_list)):
        index = f'{i+1}.0'
        textbox.insert(index, song_list[i] + '\n')  # Add a newline after each entry
    textbox.place(relx=0.5, rely=0.5, anchor=tkinter.CENTER)


button = ctk.CTkButton(root, text="SEARCH SONG", command=button_event,
                       fg_color="#EE653C",hover_color="#F66E45")
button.place(relx=0.9, rely=0.15, anchor=tkinter.CENTER)

#textbox
textbox = ctk.CTkTextbox(master=root,
                         width=500,
                         height=300,
                         fg_color=("black", "black"))


is_window_closed = False
root.protocol("WM_DELETE_WINDOW", on_close)
if(is_window_closed==True):
    exit()
root.mainloop()

1995 2
2009 2
optionmenu dropdown clicked: Depressing
button pressed
2009 Depressing
Track- Jani Na by Nachiketa Chakraborty 
1
optionmenu dropdown clicked: Sad
button pressed
2009 Sad
Track- Young Forever by JAY-Z, Mr Hudson 
Track- Two Is Better Than One (feat. Taylor Swift) by BOYS LIKE GIRLS, Taylor Swift 
2
optionmenu dropdown clicked: Happy
button pressed
2009 Happy
Track- Swapno Nil by Jeet Gannguli 
Track- Solo by Iyaz 
Track- Stand By Me by SHINee 
Track- Dora Dora by D. Imman, Balram, Kalyani Nair 
Track- Roohan by Manpreet Sandhu 
5
